# Introduction à Langchain 



### Dépendances et clé d'API

In [1]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")

# Pièce n°1: le LLM
from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Pièce n°2: le prompt
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Répond en français."), #Ici des instructions pour le modèle, 'à l'insu' de l'utilisateur
    ("human", "{question}"), # Ici la question de l'utilisateur, variable {question} à remplacer sera remplacée par la question de l'utilisateur
])

In [2]:
chatbot = prompt | llm_openai # On assemble le prompt et le LLM

## Sans mémoire

In [ ]:
print("Bienvenue dans le chatbot ! Tape 'quit' pour arrêter.")

while True:
    user_input = input("Vous: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Chatbot: Au revoir !")
        break
    print(f"User: {user_input}")
    response = chatbot.invoke({"question": user_input})
    print(f"Chatbot: {response.content}")

Bienvenue dans le chatbot ! Tape 'quit' pour arrêter.
Chatbot: content="La question semble contenir une confusion. Paris est en fait la capitale de la France. Si tu as d'autres questions sur Paris ou sur la France, n'hésite pas à demander !" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 25, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgtEPxCjDyvHvLIlYAmGiENdZxFpl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c309d3b5-ffc5-44c5-9492-e7ae0d29295c-0' usage_metadata={'input_tokens': 25, 'output_tokens': 39, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

## Avec mémoire

In [8]:
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder
promptv2 = ChatPromptTemplate.from_messages([
    ("system", "Répond en français."), #Ici des instructions pour le modèle, 'à l'insu' de l'utilisateur
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{question}"), # Ici la question de l'utilisateur, variable {question} à remplacer sera remplacée par la question de l'utilisateur
])

chatbotv2 = promptv2 | llm_openai

print("Bienvenue dans le chatbotv2, avec de la mémoire ! Tape 'quit' pour arrêter.")
chat_history = ChatMessageHistory()
while True:
    user_input = input("Vous: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Chatbot: Au revoir !")
        break
    print(f"User: {user_input}", flush=True)
    chat_history.add_user_message(user_input)
    response = chatbotv2.invoke({"question": user_input, "messages": chat_history.messages})
    print(f"Chatbot: {response.content}", flush=True)
    chat_history.add_ai_message(response.content)

Bienvenue dans le chatbotv2, avec de la mémoire ! Tape 'quit' pour arrêter.
User: Quelle est la capitale de la France ?
Chatbot: La capitale de la France est Paris.
User: Et du maroc ?
Chatbot: La capitale du Maroc est Rabat.
Chatbot: Au revoir !
